In [3]:
import pandas as pd
import numpy as np
import os
import openpyxl
from openpyxl.styles import Border, Side, PatternFill
import re

In [1]:
# файл-шаблон результатов для лазера
laser_template = r'D:\Рабочая\!_Automation\laser_specification_final_template.xlsx' 

# файл-шаблонр результаов для снабжения
supply_template = r'D:\Рабочая\!_Automation\order_supply_template.xlsx' 

In [3]:
# # словарь со ссылками
# dct =   {
#         'dxf':[],                # на файлы dxf
#          'lazer': [],            # на Excel лазер
#         'supply': [],            # на Excel снабжение
#         'bending_drawings': [],  # чертежи на гибку
#         'welding drawings': []   # чертежи на сварку
#         }  

In [4]:
# считываем таблицу со списком изделий к заказу
product_list_odr = pd.read_excel('product_list.xlsx', sheet_name='odr', names=['product_name', 'quantity', 'material'])

In [5]:
product_list_odr

,product_name,quantity,material
0,ЗРЭ-200х450,2,цинк
1,СС-6(К),3,чёрный
2,ЗВЭ-300К,4,цинк


In [6]:
# считываем таблицу с базой
product_list_base = pd.read_excel('product_list.xlsx', sheet_name='base', names=['product_name', 'dxf', 'excel_lazer', 'excel_supply', 'bending_drawings', 'welding_drawings'])

In [8]:
product_list_base.head()

,product_name,dxf,excel_lazer,excel_supply,bending_drawings,welding_drawings
0,ЗВР-400,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,NaN
1,ЗВР-300,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,NaN
2,ЗВЭ-300,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,NaN
3,ЗВЭ-200К (ЗВК-200),D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,NaN
4,ЗВЭ-300К,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,NaN


In [9]:
# мёрджим два датафрэйма, чтобы получить финальную таблицу, 
# содержащую наименования, количество, материал и ссылки на все документы
product_list_final = product_list_odr.merge(product_list_base)

In [10]:
product_list_final

,product_name,quantity,material,dxf,excel_lazer,excel_supply,bending_drawings,welding_drawings
0,ЗРЭ-200х450,2,цинк,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗР...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗР...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗР...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗР...,NaN
1,СС-6(К),3,чёрный,"D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\Самотёки, п...","D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\Самотёки, п...",NaN,NaN,"D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\Самотёки, п..."
2,ЗВЭ-300К,4,цинк,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,D:\Рабочая\САМОТЕЧНОЕ ОБОРУДОВАНИЕ\ЗАДВИЖКИ\ЗВ...,NaN


In [10]:
# функция объединения нескольких таблиц excel в одну результирующую
# def merge_excel_files(df, links_column_name, quantity_column_name, total_df):
#     '''
#     Функция объединяет excel-файлы в один по списку ссылок, которые берёт из колонки links_column_name
#         df                   - датафрэйм, из которого извлекаются ссылки и количество изделий
#         links_column_name    - название колонки со ссылками на файлы excel, строковое
#         quantity_column_name - название колонки, содержащей количество изделий, строковое
#         total_df             - название результирующего датафрэйма, строковое
#     '''
#     links_index_no    = df.columns.get_loc(links_column_name)     # получаем номер колонки
#     quantity_index_no = df.columns.get_loc(quantity_column_name)  # получаем номер колонки
    
#     locals()[total_df] = pd.DataFrame()                           # создаём пустой датафрейм с именем переданным в total_df
    
#     temp_link = row[5]
    
    
    
#     return locals()[total_df]                                     # возвращаем результирующий датафрэйм с именем переданным в total_df 

In [11]:
# создаём финальную таблицу для заявки на лазерную резку

total_lazer_list = pd.DataFrame()                              # создаём пустой датафрэйм

for row in product_list_final.itertuples():                    # итерируемся по строкам датафрэйма
    temp_link = row[5]                                         # ссылка на один из файлов-заявок excel на лазер из базы
    temp_df = pd.read_excel(temp_link,                         # считываем excel-файл во временный датафрэйм
                            skiprows=4,                        # пропускаем 4 строки
                            usecols=("B:H"),                   # отбираем нужные столбцы (отбрасываем первый с нумерацией)
                            names=['part_name',                # переименовываем колонки
                                   'quantity', 
                                   'material', 
                                   'thickness', 
                                   'bending', 
                                   'bending_quantity', 
                                   'notes'])
    multiplier = row[2]                                        # "множитель" - количество изделий
    temp_df['quantity'] = temp_df.quantity.mul(multiplier)     # умножаем кол-во деталей на кол-во изделий ("множитель") (всю колонку)
    
    total_lazer_list = pd.concat([total_lazer_list, temp_df])  # собираем всё в один датафрэйм

total_lazer_list = total_lazer_list \
                        .groupby('part_name', as_index=False) \
                        .agg({'quantity': 'sum',
                              'material':  lambda x: list(x)[0], # делаем список и берём первый элемент (все эл-ты одинаковые)
                              'thickness': lambda x: list(x)[0], 
                              'bending':   lambda x: list(x)[0], 
                              'bending_quantity': lambda x: list(x)[0], 
                              'notes':     lambda x: list(x)[0]
                             })                                      # группировка по наименованию изделий
total_lazer_list = total_lazer_list \
                        .sort_values(['material', 'thickness'])   # сортируем по материалу и толщине
total_lazer_list = total_lazer_list.reset_index(drop=True)     # сбрасываем индексы
total_lazer_list.insert(0, 'ind', total_lazer_list.index + 1)  # делаем колонку из индексов, прибавляя 1, чтоб не начинались с нуля

In [12]:
total_lazer_list

,ind,part_name,quantity,material,thickness,bending,bending_quantity,notes
0,1,"Z-1,5 пласт крышки",12,цинк,1.5,нет,NaN,NaN
1,2,"АПР-ЗВК-300-1,5 Крышка",8,цинк,1.5,нет,NaN,NaN
2,3,"ЗРЭ-200х450-1,5 крышка",4,цинк,1.5,нет,NaN,NaN
3,4,Z-2 кронштейн датчика ВПК2112,4,цинк,2.0,да,4.0,NaN
4,5,АПР-ЗВК-300-2 Козырек,32,цинк,2.0,да,1.0,NaN
5,6,АПР-ЗВК-300-2 Полоса прижимная,8,цинк,2.0,нет,NaN,NaN
6,7,ЗВК-300-2 Пыльник,8,цинк,2.0,нет,NaN,NaN
7,8,ЗРЭ-2 бегунок (новый),4,цинк,2.0,да,4.0,NaN
8,9,ЗРЭ-200х450-2 прижим,2,цинк,2.0,нет,NaN,NaN
9,10,ЗРЭ-200х450-2 пыльник,2,цинк,2.0,нет,NaN,NaN


In [13]:
def agg_to_list(x):
    '''
    Агрегирующая функция, возвращает строкой список всех элементов через запятую
    '''
    lst = set()                                   # создаём пустое множество
    for s in x:
        if str(s) != 'nan':                       # если строковый элемент не равен 'nan', то добавляем его во множество
            lst.add(s)
    if len(lst) == 0:
        return np.nan
    return str(lst).strip('{}').replace("'", "")  # превращаем множество в строку, если оно не пустое и удаляем лишние символы

In [14]:
# создаём финальную таблицу для снабжения

total_supply_list = pd.DataFrame()                                   # создаём пустой датафрэйм

for row in product_list_final.itertuples():                          # итерируемся по строкам датафрэйма
    temp_link = row[6]                                               # ссылка на один из файлов-заявок excel снабжения из базы
    if temp_link is not np.nan:
        temp_df = pd.read_excel(
                                temp_link,                           # считываем excel-файл во временный датафрэйм
                                skiprows=1,                                         # пропускаем 1 строку
                                usecols=("B:E"),                                    # отбираем нужные столбцы (отбрасываем первый с нумерацией)
                                names=['component_name',                            # переименовываем колонки
                                        'quantity', 
                                        'units', 
                                        'notes'])
        multiplier = row[2]                                          # "множитель" - количество изделий
        temp_df['quantity'] = temp_df.quantity.mul(multiplier)       # умножаем кол-во деталей на кол-во изделий ("множитель") (всю колонку)

        total_supply_list = pd.concat([total_supply_list, temp_df])  # собираем всё в один датафрэйм

total_supply_list = total_supply_list \
                        .groupby('component_name', as_index=False) \
                        .agg({'quantity': 'sum', 
                              'units': lambda x: list(x)[0],         # делаем список из единиц и берём первый элемент
                              'notes': lambda x: agg_to_list(x)      # складываем примечания из разных ячеек в одну
                             })                                      # группировка по наименованию изделий
total_supply_list = total_supply_list.reset_index(drop=True)         # сбрасываем индексы
total_supply_list.insert(0, 'ind', total_supply_list.index + 1)      # делаем колонку из индексов, прибавляя 1, чтоб не начинались с нуля

In [15]:
total_supply_list

,ind,component_name,quantity,units,notes
0,1,Конечный выключатель \nВП15К-21Б-211-54 У2.3 (...,8,шт.,NaN
1,2,Конечный выключатель \nВПК-2112-БУ2,4,шт.,NaN
2,3,"Круг д27, сталь 45",1268,мм,NaN
3,4,"Круг д28, сталь 45 калибровка",5180,мм,NaN
4,5,"Круг д37, сталь 45",152,мм,NaN
5,6,"Мотор-редуктор \nNMRV-040-40-0,25/1500",2,шт.,NaN
6,7,"Мотор-редуктор \nNMRV-040-7,5-0,25/1500",4,шт.,NaN
7,8,Подшипник 80201,100,шт.,NaN
8,9,Подшипник UCFL 204,20,шт.,NaN
9,10,"Пруток д 50,0 БрАЖ9-4\nГОСТ 1628-78",320,мм,NaN


In [14]:
# test = pd.DataFrame({'part_name': ['Изделие 1', 'Изделие 2', 'Изделие 3', 'Изделие 1'],
#                     'quantity': [1, 2, 3, 4], 
#                     'notes': [np.nan, np.nan, np.nan, np.nan]})

In [17]:
# функция создания папки
def create_folder(df_column, folder_name):
    '''
    Функция создаёт папку, проверяя условия в переданной серии: 
    хотя бы одно значение в переданной колонке датафрэйма должно быть не пустым
    df_column передаётся в формате df.column_name
    folder_name - название папки, строковое
    '''
    if df_column.isna().all() == False:                           # проверка все ли значения в колонке == NaN
        if not os.path.exists(folder_name):                       # создаём папку DXF, если она не существует
            os.makedirs(folder_name)
            print(f'Папка "{folder_name}" создана')
        else:
            print(f'Папка "{folder_name}" была создана ранее')

In [18]:
# имена папок для файлов
dxf_folder_name ='DXF'
bending_drawings_folder_name = 'Чертежи гибка PDF'
welding_drawings_folder_name = 'Чертежи сварка'

In [19]:
# создаём папку DXF
create_folder(product_list_final.dxf, dxf_folder_name)

# создаём папку Чертежи гибка PDF
create_folder(product_list_final.bending_drawings, bending_drawings_folder_name)

# создаём папку Чертежи сварка
create_folder(product_list_final.welding_drawings, welding_drawings_folder_name)

Папка "DXF" создана
Папка "Чертежи гибка PDF" создана
Папка "Чертежи сварка" создана


In [20]:
def filling_in_excel(df, template_path, ws_title=None, out_path='result.xlsx', first_row=1, first_col=1, color_pattern=None, extra=None):
    '''
    Функция заполняет шаблонный лист excel данными из датафрейма.
    В качестве параметра (color_pattern) можно передать словарь вида {'паттерн': 'цвет hex'} для 
    закрашивания ячеек выбранным цветом, если паттерн подходит.
    Параметры:
    df - датафрэйм;
    template_path - excel-файл шаблон для заполнения;
    ws_title - вкладка excel (по умолчанию =None - будет выбрана wb.active), str;
    out_path -  путь к результирующему файлу (по умолчанию ='result.xlsx');
    first_row - начальная строка заполнения (по умолчанию =1);
    first_col - начальная колонка заполнения (по умолчанию =1);
    color_pattern - словарь вида {'паттерн': 'цвет hex'} для закрашивания ячеек цветом.
    extra - словарь вида {'ячейка': 'значение'} для вставки дополнительной информации. Ячейка - в формате 'A1' и т.п.
    '''
    wb = openpyxl.load_workbook(template_path)                                   # открываем файл excel
    if ws_title == None:                                                         # указываем название вкладки, куда будем записывать
        ws = wb.active
    else:
        ws = wb[ws_title]
    
    thin = Side(border_style="thin", color="000000")                             # стиль 'thin': линии тонкие, цвет чёрный
    border = Border(top=thin, left=thin, right=thin, bottom=thin)                # параметры границ ячеек
    
    i = first_row                                                                # начальная строка заполнения
    for idx, row in df.iterrows():                                               # итерируемся по строкам
        j = first_col                                                            # начальная колонка заполнения
        for val in row:                                                          # итерируемся по всем значениям кортежа row
            ws_cell = ws.cell(row= i, column = j)                                # выбираем ячейку по номеру строки и колонки
            ws_cell.value = val                                                  # присваиваем значение ячейке
            ws_cell.border = border                                              # устанавливаем границы ячейки
            if color_pattern != None:                                            # если параметр color_pattern указан, закрашиваем цветом
                for pattern, fill_color in color_pattern.items():                # итерируемся по словарю, извлекая паттерны и цвета
                    if re.search(pattern, str(ws_cell.value)) != None:           # если нашлись совпаденяия (т.е. != None), красим в цвет
                        ws_cell.fill = PatternFill("solid", fgColor=fill_color)  # параметры заполнения цветом
            j += 1
        i += 1
    if extra != None:                                                            # итеритуемся по ключам и значениям словаря extra
        for key, value in extra.items():
            ws[key] = value                                                      # записываем значения в ячейку
    wb.save(out_path)                                                            # сохраняем результат
    wb.close() 

In [25]:
color_pattern_dic = {'чёрный': "DDDDDD", '.*рейк.*': "ffff00", '.*реек.*': "ffff00"}        # словарь вида {'паттерн': 'цвет hex'} для закрашивания ячеек

In [26]:
extra_dic = {'C2': 'номе заказа', 'B3': 'сегодняшняя дата'}

In [29]:
filling_in_excel(df=total_lazer_list, template_path=laser_template, out_path='хрень подзаборная.xlsx', first_row=6, color_pattern=color_pattern_dic, extra=extra_dic)